In [16]:
chem_text = open('exercise_experimentals.txt').read()

In [17]:
exper = [i for i in chem_text.split('\n') if i != '']

In [18]:
import spacy, re
from chemdataextractor.doc import Document, Paragraph
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

In [19]:
added = [0,1,3,4,5,6,8,9]
cooled = [10]
evacuated = [1]
dissolved = [7]

In [20]:
added_experiments = [exper[i] for i in added]

## Remove Chems

In [21]:
from nltk.tokenize.punkt import PunktSentenceTokenizer
tokenizer = PunktSentenceTokenizer()

In [22]:
def format_spans(span):
    chems = []
    for i in span:
        chems.append((i.text, i.start, i.end))
        
    return chems

def remove_chems(text, sorted_chems):

    phrases = []
    marker = 0
    chem_map = {}
    for idx, cem in enumerate(sorted_chems):
        chem_map[f'chem_{idx}'] = cem

        start_cem = cem[1]
        end_cem = cem[2]

        phrases.append(text[marker:start_cem])
        marker = end_cem

        if idx == len(sorted_chems)-1:
            phrases.append(text[marker:])

    return 'chem'.join(phrases), chem_map
    
def extract_chems(text):
    
    chems = Document(text).cems
    chems = format_spans(chems)
    sorted_chems = sorted(chems, key= lambda x: x[1])
    removed_chems, chem_map = remove_chems(text, sorted_chems)
    
    return removed_chems, sorted_chems, text, chem_map

In [23]:
def extract_cem_vols(text):
    
    chem_vol = {}

    for idx,  i in enumerate(extract_cems(text)):
        idx += 1
        chem_vol[f'chem_{idx}'] = {}

        search= re.search(f"{i} \([;×,~\+:/\w,°%.\- =]+\)", text)

        if search:

            chem, vol = search.group(0).split(' (')
            vol = f'({vol}'
            chem_vol[f'chem_{idx}']['name'] = chem
            chem_vol[f'chem_{idx}']['volume'] = vol
        else:
            chem_vol[f'chem_{idx}']['name'] = i
            chem_vol[f'chem_{idx}']['volume'] = 'unspecified'
    return chem_vol

In [24]:
def extract_cems(text):
    cems = Document(text).cems
    return [i.text for i in cems]

In [179]:
def gather_process(text):
    
    '''
    This script tries to identify the main clause. This is 
    done by finding the root of the sentences verb phrase. 
    In theory this will describe the relation shit between
    the two noun phrases which make up the experiment. 

    args:
        text: The raw text of the epxeriment

    returns:
        process: The dictionary formatted output of the experiment 
        summariser
    '''
    first_sentence = tokenizer.tokenize(text)[0]

    process={
             'method' : None}

    doc = nlp(first_sentence)

    for idx, i in enumerate(doc):
        if i.pos_ == 'VERB':
            None
        if i.dep_ == 'ROOT':
            print(i.text)
            if i.text not in ['added', 'evacuated', 'stirred']:
                continue
                
            phrase_1 = doc[:idx]

            method = i.text
            phrase_2 = doc[idx+1:]
            process['solution_1']=extract_cem_vols(doc[:idx].text)
            process['solution_2']=extract_cem_vols(doc[idx+1:].text)
            process['method']=i.text#
            process['type'] = 'Unspecified'
            break
            
    return process

In [180]:
volumes = re.compile(r"\([;×,~\+:/\w,°%.\- =]+\)")
def remove_vols(text):
    
    text_vols = []
    
    for match in re.finditer(volumes, text):
        text_vols.append((match.group(0),match.start(), match.end()))
        
    phrases = []
    marker = 0
    vol_map = {}
    for idx, vol in enumerate(text_vols):
        vol_map[f'vol_{idx}'] = vol[0]
        
        text = re.sub(vol[0], '', text)
        
    text = re.sub("\(\)", '', text)
    text = re.sub('  ', ' ', text)
    return text, vol_map

In [181]:
def format_output_text(input_text, process):
    
    for key in process:
        
        if type(process[key]) == str:
            
            if process[key] == 'Unspecified':
                continue
            
            input_text = re.sub(process[key], f"[{process[key]}]", input_text)
            
        elif type(process[key]) == dict:
            
            for subkey in process[key]:
                
                for param in process[key][subkey]:
                    
                    input_text = re.sub(process[key][subkey][param], "[{}]".format(process[key][subkey][param]), input_text)
    
    input_text = re.sub('\)\]\)', ')]', input_text)
    input_text = re.sub('\(\[\(', '[(', input_text)
    process['annotated'] = input_text
    return process, input_text

In [182]:
def experiment_summariser(experiment):
    processes = []
    sentences = tokenizer.tokenize(experiment)

    for sentence in sentences:
        test_sentence = sentence # main info
        test_text, vols = remove_vols(test_sentence)
    
        process = gather_process(test_text)
        processes.append(process)
    
    return process, processes

In [183]:
# for idx, ex in enumerate(added_experiments):
    
#     process = experiment_summariser(ex)
    
#     with open('output/experiment_{}.json'.format(idx), 'w') as file:
        
#         json.dump(process, file)
    
    

In [184]:
process, processes = experiment_summariser(added_experiments[0])

added
removed
removed


In [185]:
processes

[{'method': 'added',
  'solution_1': {'chem_1': {'name': 'potassium osmate dihydrate',
    'volume': 'unspecified'},
   'chem_2': {'name': '4-Methylmorpholine N-oxide', 'volume': 'unspecified'}},
  'solution_2': {'chem_1': {'name': '‐5‐butyl‐1H,3H,5H,7aH‐pyrrolo[1,2‐c][1,3]oxazol‐3‐one',
    'volume': 'unspecified'},
   'chem_2': {'name': 'H2O', 'volume': 'unspecified'},
   'chem_3': {'name': 'acetone', 'volume': 'unspecified'}},
  'type': 'Unspecified'},
 {'method': None},
 {'method': None}]

In [124]:
added_experiments[0]

'4-Methylmorpholine N-oxide (1.76 mL, 8.42 mmol) and potassium osmate dihydrate (97.3 mg, 0.38 mmol) were added to a solution of (5R,7aS)‐5‐butyl‐1H,3H,5H,7aH‐pyrrolo[1,2‐c][1,3]oxazol‐3‐one (347 mg, 1.91 mmol) in acetone/H2O (2:1, 54 mL). After the mixture was stirred at room temperature for 5 h, the solvents were removed under reduced pressure. Traces of water were removed from the mixture by azeotropic distillations with toluene (2 x 5 mL) under reduced pressure and the residue was purified by silica gel column chromatography (hexane/EtOAc = 3/1) to give (5S,6S,7R,7aS)-5-Butyl-6,7-dihydroxytetrahydropyrrolo[1,2-c]oxazol-3(1H)-one (398 mg, 97%) as brown oil.'

In [26]:
removed_chems, sorted_chems, text, chem_map = extract_chems(added_experiments[0])
removed_vols, vol_map = remove_vols(removed_chems)

In [28]:
process = gather_process(added_experiments[0])

In [58]:
out, input_text= format_output_text(added_experiments[3], process)

In [44]:
#def test_output(output_process, gold_output):
    
#     score = 0 
#     print(gold_output)
#     for key in output_process:
#         print(key)
#         if output_process[key] == gold_output[key]:
#             score += 1
            
#     return score

    

In [45]:
import Levenshtein
from Levenshtein import distance
from fuzzywuzzy import fuzz

## test_output(process, experiment_0)

## Remove Volumes


In [7]:
## Phase 1 Extract Phrases

In [15]:
exp_summ = experiment_summariser(exper[2])

In [16]:
exp_summ

{'method': 'evacuated',
 'solution_1': {'chem_1': {'Name': 'nitrogen', 'Volume': 'Unspecified'}},
 'solution_2': {'chem_1': {'Name': 'nitrogen', 'Volume': 'Unspecified'}},
 'type': 'Unspecified'}

In [17]:
volumes = re.compile(r"\([;×,~\+:/\w,°%.\- =]+\)")

def replace_chems_n_vols(text):
    
    chems = [i.text for i in Document(text).cems]
    
    for c in chems:
        text= re.sub(c, 'chem', text)
        
    text = re.sub(volumes, '', text)
    
    return text


text=replace_chems(exper[0])

print(text)

NameError: name 'replace_chems' is not defined

In [18]:
# def decode_experiment(summ):
    
#     if summ['Method'] == 'added':
        
#         solution = []
#         volumes = []
#         for chem in summ['solution_1']:
                        
#             if summ['solution_1'][chem]['Volume'] != 'Unspecified':
#                 solution.append(f"{summ['solution_1'][chem]['Name']} {summ['solution_1'][chem]['Volume']}")
#             else:
#                 solution.append(f"{summ['solution_1'][chem]['Name']}")
#                 #volumes.append(summ['solution_1'][chem]['Volume'])
        
#         solution_2 = []
        
#         for chem in summ['solution_2']:
            
#             if summ['solution_2'][chem]['Volume'] != 'Unspecified':
#                 solution_2.append(f"{summ['solution_2'][chem]['Name']} {summ['solution_2'][chem]['Volume']}")
#             else:
#                 solution_2.append(f"{summ['solution_2'][chem]['Name']}")
        
#         if len(solution) >= 2:
#             solution = ' and '.join(solution)
#         else:
#             solution = solution[0]
            
#         if len(solution_2) >= 2:
#             solution_2 = ' and '.join(solution_2)
#         else:
#             solution_2 = solution_2[0]
            
#         return f"{solution} was added to {solution_2}"

In [142]:
# decode_experiment(exp_summ)

In [151]:
sentences=tokenizer.tokenize(exper[0])

In [71]:
gold = json.load(open('gold_output/experiment_0_gold.json'))

In [189]:
test_10 = {'idx': 10,
 'method': 'cooled',
 'solution_1': {'chem_1': {'name': 'NaH',
   'volume': '(2.12 g, 52.9 mmol, 1.20 equiv, 60% in mineral oil)'},
               'chem_2': {'name': 'THF/DMSO',
   'volume': '(10:1, 150 mL)'}},
                
                
    'solution_2': {'chem_1': {'name': 'tributyl(iodomethyl)stannane',
   'volume': '(19.0 g, 44.1 mmol, 1.00 equiv)'}
                  },
 'type': 'continous'}
          
with open('gold_output/experiment_10_gold.json', 'w') as path:
    json.dump(test_10, path)

In [340]:

    
sentences=tokenizer.tokenize(exper[0])
doc = nlp(sentences[0])

for idx, token in enumerate(doc):
    #print(token, token.dep_)
    
    
    if token.dep_ in ignore_list:
        doc=nlp(' '.join([i for i in sentences[0].split(' ') if i != token.text ]))
        
    if token.pos_ == 'VERB' or token.dep_ == 'ROOT':
        method = token.text
        
        clause_1 = doc[:idx-1]
        clause_2 = doc[idx:]

In [341]:
sentences[0]

'4-Methylmorpholine N-oxide (1.76 mL, 8.42 mmol) and potassium osmate dihydrate (97.3 mg, 0.38 mmol) were added to a solution of (5R,7aS)‐5‐butyl‐1H,3H,5H,7aH‐pyrrolo[1,2‐c][1,3]oxazol‐3‐one (347 mg, 1.91 mmol) in acetone/H2O (2:1, 54 mL).'

In [337]:
clause_1

4-Methylmorpholine N-oxide (1.76 mL, 8.42 mmol) and potassium osmate dihydrate (97.3 mg, 0.38 mmol)

In [338]:
method

'added'

In [339]:
clause_2

to a solution of (5R,7aS)‐5‐butyl‐1H,3H,5H,7aH‐pyrrolo[1,2‐c][1,3]oxazol‐3‐one (347 mg, 1.91 mmol) in acetone/H2O (2:1, 54 mL).

In [287]:
def clause_breakdown(clause):
    
    for token in clause:
        
        if token.dep_ == 'cc':
            
            nps = clause.text.split(token.text)
            break
            
    return nps

In [288]:
nps_1 = clause_breakdown(clause_1)

In [ ]:
def